In [36]:
import torch
from torch import nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(in_features=2, out_features=1)
        
    def forward(self, x):
        return self.layer(x)

In [37]:
from safetensors.torch import save_file

torch.manual_seed(42)
model = MyModel()
save_file(model.state_dict(), "model.safetensors")
torch.save(model.state_dict(), "model.pt")
model.state_dict()


OrderedDict([('layer.weight', tensor([[0.5406, 0.5869]])),
             ('layer.bias', tensor([-0.1657]))])

In [38]:
from safetensors.torch import load_file

test_model = MyModel() 

weights = load_file("./model.safetensors")

weights

{'layer.bias': tensor([-0.1657]), 'layer.weight': tensor([[0.5406, 0.5869]])}

In [39]:
test_model.load_state_dict(weights)

<All keys matched successfully>

In [40]:
test_model.state_dict()

OrderedDict([('layer.weight', tensor([[0.5406, 0.5869]])),
             ('layer.bias', tensor([-0.1657]))])

In [41]:
from safetensors import safe_open

with safe_open("model.safetensors", framework="pt") as f:
    for i in f.keys():
        print(f"Name : {i}  and tensor: {f.get_tensor(i)}")
    

Name : layer.bias  and tensor: tensor([-0.1657])
Name : layer.weight  and tensor: tensor([[0.5406, 0.5869]])


In [42]:
from safetensors import safe_open

with safe_open("model.safetensors", framework="pt") as f:
    print(f.get_tensor("layer.weight"))
    
    

tensor([[0.5406, 0.5869]])


In [49]:
import os
import datetime
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import torch

start_st = datetime.datetime.now()
weights = load_file("./model.safetensors", device="cpu")
load_time_st = datetime.datetime.now() - start_st
print(f"Loaded safetensors {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load("./model.pt", map_location="cpu")
load_time_pt = datetime.datetime.now() - start_pt
print(f"Loaded pytorch {load_time_pt}")

print(f"on CPU, safetensors is faster than pytorch by: {load_time_pt/load_time_st:.1f} X")

Loaded safetensors 0:00:00.000981
Loaded pytorch 0:00:00.002020
on CPU, safetensors is faster than pytorch by: 2.1 X


In [50]:
from torch import nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Linear(100, 100)
        self.b = self.a

    def forward(self, x):
        return self.b(self.a(x))


model = Model()
print(model.state_dict())
# odict_keys(['a.weight', 'a.bias', 'b.weight', 'b.bias'])
torch.save(model.state_dict(), "model.bin")
# This file is now 41k instead of ~80k, because A and B are the same weight hence only 1 is saved on disk with both `a` and `b` pointing to the same buffer

OrderedDict({'a.weight': tensor([[-0.0487,  0.0587,  0.0882,  ..., -0.0726,  0.0023, -0.0683],
        [-0.0848, -0.0551, -0.0875,  ...,  0.0390, -0.0050, -0.0603],
        [-0.0612, -0.0896, -0.0326,  ..., -0.0479, -0.0814,  0.0839],
        ...,
        [ 0.0407,  0.0201,  0.0366,  ...,  0.0722,  0.0103,  0.0911],
        [ 0.0532,  0.0534, -0.0858,  ..., -0.0748,  0.0761, -0.0996],
        [-0.0037, -0.0617, -0.0421,  ..., -0.0585,  0.0528, -0.0033]]), 'a.bias': tensor([-0.0918,  0.0981,  0.0261,  0.0920,  0.0125,  0.0143,  0.0721, -0.0680,
         0.0944, -0.0643,  0.0091, -0.0650, -0.0663,  0.0770, -0.0264, -0.0188,
         0.0781,  0.0980, -0.0902, -0.0017,  0.0286,  0.0960, -0.0399, -0.0325,
        -0.0586,  0.0256,  0.0247, -0.0016, -0.0101, -0.0696, -0.0411, -0.0380,
         0.0263,  0.0442,  0.0467, -0.0763, -0.0077, -0.0191,  0.0997,  0.0722,
        -0.0746, -0.0549, -0.0755, -0.0250, -0.0875,  0.0470, -0.0441, -0.0502,
         0.0047, -0.0320,  0.0755,  0.0906, -0.089

In [52]:
sum(param.numel() for param in model.parameters())

10100

In [53]:
from huggingface_hub import get_safetensors_metadata

metadata = get_safetensors_metadata("bigscience/bloomz-560m")
metadata
metadata.files_metadata["model.safetensors"].metadata

metadata = get_safetensors_metadata("bigscience/bloom")


metadata
len(metadata.files_metadata)

get_safetensors_metadata("runwayml/stable-diffusion-v1-5")

Parse safetensors files: 100%|██████████| 72/72 [00:04<00:00, 16.60it/s]


NotASafetensorsRepoError: 'runwayml/stable-diffusion-v1-5' is not a safetensors repo. Couldn't find 'model.safetensors.index.json' or 'model.safetensors' files.